<a href="https://colab.research.google.com/github/andrea212023/ML-Kis/blob/main/HW_2_4_KNN_Cross_validation_and_tuning_of_hyperparameters_KIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [ ]:
!pip install opendatasets --upgrade --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [ ]:
import opendatasets as od
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from process_bank_churn import preprocess_data, preprocess_new_data, main_process_data

In [ ]:
dataset_url = 'https://www.kaggle.com/competitions/bank-customer-churn-prediction-dlu/'
od.download(dataset_url)
data_dir = './bank-customer-churn-prediction-dlu'
raw_df = pd.read_csv('./bank-customer-churn-prediction-dlu/train.csv')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: andriannakis
Your Kaggle Key: ··········


100%|██████████| 637k/637k [00:00<00:00, 1.05MB/s]


Extracting archive ./bank-customer-churn-prediction-dlu/bank-customer-churn-prediction-dlu.zip to ./bank-customer-churn-prediction-dlu


In [6]:
# Excluded columns
excluded_cols = ['CustomerId', 'CreditScore', 'Tenure', 'HasCrCard', 'EstimatedSalary', 'Surname', 'id']
# Preprocessing
datasets = preprocess_data(raw_df, excluded_cols)

In [7]:
print("train_X shape:", datasets['train_X'].shape)
print("train_y shape:", datasets['train_y'].shape)
print("val_X shape:", datasets['val_X'].shape)
print("val_y shape:", datasets['val_y'].shape)
print("Input columns:", datasets['input_cols'])
print("Scaler:", datasets['scaler'])
print("Encoder:", datasets['encoder'])

# Display preprocessed data
print(datasets['train_X'].head())

train_X shape: (12000, 9)
train_y shape: (12000,)
val_X shape: (3000, 9)
val_y shape: (3000,)
Input columns: ['Age', 'Balance', 'NumOfProducts', 'IsActiveMember', 'Geography_France', 'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male']
Scaler: StandardScaler()
Encoder: OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        Age   Balance  NumOfProducts  IsActiveMember  Geography_France  \
0 -0.944682  1.475464      -1.108193        1.016977               1.0   
1  0.161227  2.254434      -1.108193       -0.983306               1.0   
2 -0.330288  1.223831       0.769304       -0.983306               0.0   
3  2.495924 -0.717950       0.769304        1.016977               0.0   
4 -2.050591 -0.717950       0.769304        1.016977               1.0   

   Geography_Germany  Geography_Spain  Gender_Female  Gender_Male  
0                0.0              0.0            0.0          1.0  
1                0.0              0.0            1.0          0.0  
2 

1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [8]:
# Implement KNN algorithm
from sklearn.neighbors import KNeighborsClassifier

X_train = datasets['train_X']
y_train = datasets['train_y']
X_val = datasets['val_X']
y_val = datasets['val_y']

# Train KNN
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

# Predict on train and validation datasets
y_train_pred_proba = knn.predict_proba(X_train)[:, 1]
y_val_pred_proba = knn.predict_proba(X_val)[:, 1]

# Compute ROC AUC
roc_auc_train = roc_auc_score(y_train, y_train_pred_proba)
roc_auc_val = roc_auc_score(y_val, y_val_pred_proba)

# Print AUROC
print(f"AUROC for train dataset: {roc_auc_train}")
print(f"AUROC for validation dataset: {roc_auc_val}")


AUROC for train dataset: 0.9470605471076281
AUROC for validation dataset: 0.8906228136360519


**Conclusions:**

1. The AUROC value for the training set (0.947) is significantly higher than for the validation set (0.891), indicating some level of overfitting.
2. Therefore, it can be concluded that the model has high variance, which is a sign of overfitting.
3. The model fits the training data very well but does not perform as well on the validation data.

2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [10]:
# Implement GridSearchCV algorithm
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15]
}

# Find best params
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print best params
best_params = grid_search.best_params_
print(f"Best params: {best_params}")

Best params: {'n_neighbors': 15}


In [11]:
# Save best model
knn_best = grid_search.best_estimator_
print (knn_best)

KNeighborsClassifier(n_neighbors=15)


In [12]:
# Predict on train and validation datasets
y_train_pred_proba = knn_best.predict_proba(X_train)[:, 1]
y_val_pred_proba = knn_best.predict_proba(X_val)[:, 1]

# Compute ROC AUC
roc_auc_train_best = roc_auc_score(y_train, y_train_pred_proba)
roc_auc_val_best = roc_auc_score(y_val, y_val_pred_proba)

# Print AUROC for train and validation datasers
print(f"AUROC for train dataset: {roc_auc_train_best}")
print(f"AUROC for validation dataset: {roc_auc_val_best}")

AUROC for train dataset: 0.9469101656013145
AUROC for validation dataset: 0.9199406680842308


**Conclusions:**

1. The optimization of the n_neighbors hyperparameter using GridSearchCV improved the quality of the KNN model.

AUROC for the training set: 0.9469 (with optimal parameters).

AUROC for the validation set: 0.9199 (with optimal parameters).

2. The model's quality improved compared to the previous kNN version (default parameters), where the AUROC for the validation set was 0.8906.

3. Compared to the decision tree, the kNN model with optimal parameters showed slightly better results on the validation set (0.9199 vs. 0.9167).

3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_seed`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [15]:
# Implement the model with given params
import time

param_grid = {
    'max_depth': range(1, 21, 2),
    'max_leaf_nodes': range(2, 11, 1)
}

dt = DecisionTreeClassifier(random_state=42)

# Restart GridSearchCV
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=3, scoring='roc_auc')

# Measure time
start_time = time.time()
grid_search.fit(X_train, y_train)
end_time = time.time()

# Print best params
best_params = grid_search.best_params_
print(f"Best params: {best_params}")

Best params: {'max_depth': 5, 'max_leaf_nodes': 10}


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [16]:
# Save best model
dt_best = grid_search.best_estimator_

# Predict AUROC on train and validation datasets
roc_auc_train_best = roc_auc_score(y_train, dt_best.predict_proba(X_train)[:, 1])
roc_auc_val_best = roc_auc_score(y_val, dt_best.predict_proba(X_val)[:, 1])

# Print AUROC
print(f"AUROC for train data: {roc_auc_train_best}")
print(f"AUROC for validation data: {roc_auc_val_best}")

# Print time
print(f"Time for searching best params: {end_time - start_time} s")

AUROC for train data: 0.9014754782174744
AUROC for validation data: 0.9002184649152891
Time for searching best params: 5.066710472106934 s


**Conclusions:**

The model with manually tuned parameters for the DecisionTreeClassifier demonstrated better performance on both the training and validation datasets compared to the model found using GridSearchCV. Therefore, the manual parameter tuning yielded better results in this specific case.

4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [19]:
# Implement RandomizedSearchCV algorithm
from sklearn.model_selection import RandomizedSearchCV

In [20]:
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

In [21]:
dt = DecisionTreeClassifier(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=dt,
    param_distributions=params_dt,
    n_iter=40,
    scoring='roc_auc',
    cv=3,
    random_state=42
)

# Measure time
start_time = time.time()
random_search.fit(X_train, y_train)
end_time = time.time()

# Print best params
best_params = random_search.best_params_
print(f"Best params: {best_params}")

Best params: {'splitter': 'best', 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_leaf_nodes': 14, 'max_features': None, 'max_depth': 16, 'criterion': 'entropy'}


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [22]:
# Save best model
dt_random_search_best = random_search.best_estimator_

# Predict AUROC on train and validation datasets
roc_auc_train_best = roc_auc_score(y_train, dt_random_search_best.predict_proba(X_train)[:, 1])
roc_auc_val_best = roc_auc_score(y_val, dt_random_search_best.predict_proba(X_val)[:, 1])

# Print AUROC
print(f"AUROC for train data: {roc_auc_train_best}")
print(f"AUROC for validation data: {roc_auc_val_best}")

# Print time
print(f"Time for searching best params: {end_time - start_time} s")

AUROC for train data: 0.9169275635848141
AUROC for validation data: 0.9166204815145071
Time for searching best params: 1.3394103050231934 s


**Observations:**

1. RandomizedSearchCV found a model with a maximum depth of 16, indicating that a deeper tree might learn better from the data.
2. In comparison, GridSearchCV had a maximum depth of only 5.
3. RandomizedSearchCV selected max_leaf_nodes = 14, which is greater than the max_leaf_nodes = 10 found by GridSearchCV, suggesting a more complex tree structure.
4. RandomizedSearchCV used the entropy criterion, while GridSearchCV opted for gini. This suggests that entropy may be a more suitable criterion for this dataset.
5. RandomizedSearchCV also fine-tuned other parameters, such as min_samples_split and min_samples_leaf, allowing for more precise adjustments to the model for better performance.

**Conclusions:**
The model found through RandomizedSearchCV achieved better AUROC scores on both the training and validation datasets compared to the model found using GridSearchCV. This suggests that the wider range of parameters and more comprehensive search space used by RandomizedSearchCV helped identify more optimal hyperparameter settings for the decision tree.

**When comparing RandomizedSearchCV with the manually tuned model:**

RandomizedSearchCV achieved a slightly lower AUROC on the training dataset and a marginally lower AUROC on the validation dataset.
This indicates that both approaches found reasonably good configurations, but the manually tuned model performed slightly better overall.

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

In [23]:
test_df = pd.read_csv('./bank-customer-churn-prediction-dlu/test.csv')
test_df.head(5)

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15000,15594796.0,Chu,584.0,Germany,Male,30.0,2.0,146053.66,1.0,1.0,1.0,157891.86
1,15001,15642821.0,Mazzi,551.0,France,Male,39.0,5.0,0.00,2.0,1.0,1.0,67431.28
2,15002,15716284.0,Onyekachi,706.0,France,Male,43.0,8.0,0.00,2.0,1.0,0.0,156768.45
3,15003,15785078.0,Martin,717.0,Spain,Male,45.0,3.0,0.00,1.0,1.0,1.0,166909.87
4,15004,15662955.0,Kenechukwu,592.0,Spain,Male,43.0,8.0,0.00,2.0,1.0,1.0,143681.97


In [24]:
# Exclusion of columns
input_cols = datasets['input_cols']
scaler = datasets['scaler']
encoder = datasets['encoder']

# Preprocessing
X_test = preprocess_new_data(test_df, input_cols, scaler, encoder, excluded_cols)

# Manual search of hyperparameters
best_clf_manual = DecisionTreeClassifier(random_state=42, max_depth=7)
best_clf_manual.fit(datasets['train_X'], datasets['train_y'])

# Predict on train and validation datasets
y_train_pred_proba_manual = best_clf_manual.predict_proba(datasets['train_X'])[:, 1]
y_val_pred_proba_manual = best_clf_manual.predict_proba(datasets['val_X'])[:, 1]

# Compute AUROC on train and validation datasets
roc_auc_train_manual = roc_auc_score(datasets['train_y'], y_train_pred_proba_manual)
roc_auc_val_manual = roc_auc_score(datasets['val_y'], y_val_pred_proba_manual)

# Predict on test dataset
predictions_manual = best_clf_manual.predict(X_test)
predictions_proba_manual = best_clf_manual.predict_proba(X_test)[:, 1]

# Print AUROC for manual search of hyperparameters
print(f"AUROC for train dataset (manual): {roc_auc_train_manual}")
print(f"AUROC for validation dataset (manual): {roc_auc_val_manual}")


AUROC for train dataset (manual): 0.9393145713767181
AUROC for validation dataset (manual): 0.9238754372727898


In [30]:
# RandomizedSearchCV model
best_clf_random = dt_random_search_best
best_clf_random.fit(datasets['train_X'], datasets['train_y'])

# Predict on train and validation datasets
y_train_pred_proba_random = best_clf_random.predict_proba(datasets['train_X'])[:, 1]
y_val_pred_proba_random = best_clf_random.predict_proba(datasets['val_X'])[:, 1]

# Compute AUROC on train and validation datasets
roc_auc_train_random = roc_auc_score(datasets['train_y'], y_train_pred_proba_random)
roc_auc_val_random = roc_auc_score(datasets['val_y'], y_val_pred_proba_random)

# Predict on test dataset
predictions_random = best_clf_random.predict(X_test)
predictions_proba_random = best_clf_random.predict_proba(X_test)[:, 1]

# Print AUROC for RandomizedSearchCV
print(f"AUROC for train dataset (RandomizedSearchCV): {roc_auc_train_random}")
print(f"AUROC for validation dataset (RandomizedSearchCV): {roc_auc_val_random}")

AUROC for train dataset (RandomizedSearchCV): 0.9169275635848141
AUROC for validation dataset (RandomizedSearchCV): 0.9166204815145071


In [26]:
# Save results for both models
submission_df_manual = pd.read_csv('./bank-customer-churn-prediction-dlu/sample_submission.csv')
submission_df_manual['Exited'] = predictions_proba_manual
submission_df_manual.to_csv('drive/MyDrive/Colab Notebooks/data/submission_manual.csv', index=False)

submission_df_random = pd.read_csv('./bank-customer-churn-prediction-dlu/sample_submission.csv')
submission_df_random['Exited'] = predictions_proba_random
submission_df_random.to_csv('drive/MyDrive/Colab Notebooks/data/submission_random.csv', index=False)

In [27]:
submission_df_manual.head()

,id,Exited
0,15000,0.167969
1,15001,0.021478
2,15002,0.055556
3,15003,0.714286
4,15004,0.037433


In [28]:
submission_df_random.head()

,id,Exited
0,15000,0.237911
1,15001,0.012115
2,15002,0.203947
3,15003,0.569848
4,15004,0.082171


**Conclusions:**

1. Manual hyperparameter tuning produced better results on the validation dataset.
2. While RandomizedSearchCV also performed well, its results were slightly inferior, likely because manual tuning was more tailored to the specific dataset.
3. RandomizedSearchCV identified different hyperparameters that yielded good but not optimal results.
4. This suggests that manual tuning can be highly effective for specific datasets, while RandomizedSearchCV offers a faster, automated approach to finding reasonably good parameters with less manual effort.